# Building an ETL Pipeline
---

### Step 0: Install the required python packages

In [ ]:
pip install --upgrade sodapy

In [ ]:
pip install --upgrade db-dtypes

In [ ]:
pip install --upgrade pyarrow

In [ ]:
pip install --upgrade google-cloud-bigquery

#### Now, on the top of your Notebook select "Kernel" -> "Restart and Clear Output"
Then, continue from the next cell

### Step 1: Setup your NYC Open Data variables (ACTION REQUIRED HERE)

In [ ]:
# import libraries
import pandas as pd
import numpy as np
from sodapy import Socrata
from google.cloud import bigquery
from google.oauth2 import service_account

In [ ]:
# setup the host name for the API endpoint (the https:// part will be added automatically)
# only need to change this if you are not using NYC Open Data
data_url = 'data.cityofnewyork.us'

In [ ]:
# setup the data set at the API endpoint (311 data in this case)
# For example: https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9.json
# would give us 'erm2-nwe9'

# https://data.cityofnewyork.us/resource/qcdj-rwhu.json
sidewalk_dataset = 'qcdj-rwhu'

# https://data.cityofnewyork.us/resource/w7w3-xahh.json
businesses_dataset = 'w7w3-xahh'

In [ ]:
# Setup your App Token, which you created in Week 6
# You can find your app token by logging into: https://data.cityofnewyork.us/profile/edit/developer_settings
app_token = 'your app token here'

In [ ]:
# run this cell to setup your Socrata client that connects python to NYC Open Data

# create the client that points to the API endpoint
nyc_open_data_client = Socrata(data_url, app_token, timeout = 200)
print(f"nyc open data client name is: {nyc_open_data_client}")
print(f"nyc open data client data type is: {type(nyc_open_data_client)}")

### Step 2: Setup your Google BigQuery variables (ACTION REQUIRED HERE)

If you did not create a key path in class on 3/30/22 (which created a json file on your computer), you must create one to continue:
1. Open BigQuery
2. On the top-left, click on the Navigation Menu
3. In the Navigation Menu, go to "IAM & Admin" -> "Sercive Accounts"
4. On the top of the page, click on "Create Service Account"
5. Account name: cis9440-spring2022
6. Click create and continue
7. Set Role to Owner
8. Click Continue
9. Click Done
10. In the new row for your Service Account, click on the 3 dots in the "Action" column. Select "Manage Keys"
11. Click "Add Key", then "Create New Key". Select the "JSON" radio button and click "Create"
12. In the next cell, set key_path to the exact file path of your new JSON file. For example, it will look like r'C:\Users\Downloads\cis9440-324315-70048a5e1138.json'

In [ ]:
# CHANGE THIS TO YOUR FILE PATH
key_path = r'your json key path here'

In [ ]:
# run this cell without changing anything to setup your credentials
credentials = service_account.Credentials.from_service_account_file(key_path,
                                                                    scopes=["https://www.googleapis.com/auth/cloud-platform"],)
bigquery_client = bigquery.Client(credentials = credentials,
                                 project = credentials.project_id)

print(f"bigquery client name is: {bigquery_client}")
print(f"bigquery client data type is: {type(bigquery_client)}")

Now, you need to create your dataset id:
1. Go to bigquery
2. Inside the "Explorer" window, click on the 3 dots to the right of your cis9440 project called "View Actions"
3. Select "Create dataset"
4. Leave the Project ID as it is, name your Dataset ID etl_dataset
5. Expand your cis9440 project with the triangle on its left-hand side so you can see your new etl_dataset dataset
6. On the right of your etl_dataset, click the 3 dots for "View Actions" -> "Open"
7. You should now see the "Dataset info". Copy the entire "Dataset ID" and paste it in the variable below

In [ ]:
dataset_id = 'your dataset id here'   # PASTE THIS DATASET ID FROM ABOVE STEPS

dataset_id = dataset_id.replace(':', '.')
print(f"your dataset_id is: {dataset_id}")

### Step 3: Extract data

1. connect to NYC Open Data with API Key
2. pull specific dataset as a pandas dataframe
3. Look at shape of extracted data

#### sodapy client.get parameters
1. select
2. where
3. order
4. limit
5. group

In [ ]:
# Get the total number of records in our the entire data set
for ds in [sidewalk_dataset, businesses_dataset]:
    total_record_count = nyc_open_data_client.get(ds, select = "COUNT(*)")
    print(f"total records in {ds}: {total_record_count[0]['COUNT']}")

In [ ]:
# Now, loop through target data set to pull all rows in chunks (we cannot pull all rows at once)
# AGAIN, UPDATE WHERE FILTER INSIDE BELOW FUNCTION

def extract_socrata_data(data_set,
                         chunk_size = 2500,
                         where = None):
    
    # measure time this function takes
    import time
    start_time = time.time()
    
    # get total number or records
    if where == None:
        total_records = int(nyc_open_data_client.get(data_set,
                                                     select= "COUNT(*)")[0]["COUNT"])
    else:
        total_records = int(nyc_open_data_client.get(data_set,
                                                     where = where,
                                                     select= "COUNT(*)")[0]["COUNT"])
    
    # start at 0, empty list for results
    start = 0                   
    results = []                

    while True:

        if where == None:
            # fetch the set of records starting at 'start'
            results.extend(nyc_open_data_client.get(data_set,
                                                    offset = start,
                                                    limit = chunk_size))
            
        elif where != None:
            results.extend(nyc_open_data_client.get(data_set,
                                                    where = where,
                                                    offset = start,
                                                    limit = chunk_size))
        # update the starting record number
        start = start + chunk_size

        # if we have fetched all of the records (we have reached total_records), exit loop
        if (start > total_records):
            break

    # convert the list into a pandas data frame
    data = pd.DataFrame.from_records(results)

    end_time = time.time()
    print(f"function took {round(end_time - start_time, 1)} seconds")

    print(f"the shape of your dataframe is: {data.shape}")
    return data

In [ ]:
sidewalk_df = extract_socrata_data(chunk_size = 2500,
                         data_set = sidewalk_dataset,
                         where = None)

In [ ]:
businesses_df = extract_socrata_data(chunk_size = 2500,
                         data_set = businesses_dataset,
                         where = None)

In [ ]:
businesses_df.columns

In [ ]:
sidewalk_df.columns

In [ ]:
data = sidewalk_df.merge(businesses_df,
                         how = 'inner',
                         left_on = "license_nbr",
                         right_on = "license_nbr",
                         suffixes=('', '_y'))

### Step 4: Data Profiling

1. Distinct values per column
2. Null values per column
3. Summary statistics per numeric column

In [ ]:
# what are the columns in our dataframe?
data.columns

In [ ]:
# subset for only needed columns
data = data[['business_name',
             'business_name2',
             'latitude',
             'longitude',
             'street',
             'zip',
             'address_borough',
             'app_status',
             'swc_type',
             'swc_sq_ft',
             'swc_tables',
             'swc_chairs']]

In [ ]:
# create and run a function to ceate data profiling dataframe

def create_data_profiling_df(data):
    
    # create an empty dataframe to gather information about each column
    data_profiling_df = pd.DataFrame(columns = ["column_name",
                                                "column_type",
                                                "unique_values",
                                                "duplicate_values",
                                                "null_values",
                                                "non_null_values"])

    # loop through each column to add rows to the data_profiling_df dataframe
    for column in data.columns:

        info_dict = {}

        try:
            info_dict["column_name"] = column
            info_dict["column_type"] = data[column].dtypes
            info_dict["unique_values"] = len(data[column].unique())
            info_dict["duplicate_values"] = data[column].count() - len(data[column].dropna().unique())
            info_dict["null_values"] = data[column].isna().sum()
            info_dict["non_null_values"] = data[column].count()

        except:
            print(f"unable to read column: {column}, you may want to drop this column")

        data_profiling_df = data_profiling_df.append(info_dict, ignore_index=True)

    data_profiling_df.sort_values(by = ['unique_values', "non_null_values"],
                                  ascending = [False, False],
                                  inplace=True)
    
    return data_profiling_df

In [ ]:
# view your data profiling dataframe
data_profiling_df = create_data_profiling_df(data)
data_profiling_df

### Step 5: Data Cleansing

1. drop unneeded columns
2. drop duplicate rows
3. check for outliers

In [ ]:
# Run this to look at a list of your columns
data.info()

In [ ]:
# address null values
data['business_name2'].fillna(data['business_name'], inplace=True)

In [ ]:
# find number of duplicate rows
print(f"number of duplicate rows: {len(data[data.duplicated()])}")

In [ ]:
# drop duplicate rows based on entire row
data = data.drop_duplicates(keep = 'first')

# Or, based on a subset of rows, uncomment below and adjust accordingly
## data = data.drop_duplicates(subset = ["subset column"], keep = 'first')
## data = data.drop_duplicates(subset = ["subset column 1", "subset column 2"], keep = 'first')

print(f"number of rows after duplicates dropped: {len(data)}")

In [ ]:
# update columns types
for column in ['latitude',
              'longitude',
              'zip',
              'swc_sq_ft',
              'swc_tables',
              'swc_chairs']:
    try:
        data[column] = data[column].astype(int)
        
    except:
        data[column] = data[column].astype(float)
        
data.info()

### Step 4: Create Location Dimension

In [ ]:
# first, copy the entire table
location_dim = data.copy()

In [ ]:
location_dim.columns

In [ ]:
# second, subset for only the wanted columns in the dimension
location_dim = location_dim[['latitude', 'longitude', 'street',
       'zip', 'address_borough']]

In [ ]:
# third, drop duplicate rows in dimension
unique_row = ['latitude', 'longitude']
location_dim = location_dim.drop_duplicates(subset = unique_row, keep = 'first')
location_dim = location_dim.reset_index(drop = True)
location_dim

In [ ]:
# fourth, add a surrogate key
location_dim.insert(0, 'location_id', range(1, 1 + len(location_dim)))
location_dim

In [ ]:
# fifth, add the surrogate key to the data table
data = data.merge(location_dim,
                  left_on = unique_row,
                  right_on = unique_row,
                  how = 'left')

data.head(100)

### Step 5: Create Business Dimension

In [ ]:
# first, copy the entire table
business_dim = data.copy()

In [ ]:
business_dim.columns

In [ ]:
# second, subset for only the wanted columns in the dimension
business_dim = business_dim[['business_name', 'business_name2', 'swc_type', 'app_status']]

In [ ]:
# third, drop duplicate rows in dimension
unique_row = ['business_name', 'business_name2']
business_dim = business_dim.drop_duplicates(subset = unique_row, keep = 'first')
business_dim = business_dim.reset_index(drop = True)
business_dim

In [ ]:
# fourth, add a surrogate key
business_dim.insert(0, 'business_id', range(1000, 1000 + len(business_dim)))
business_dim

In [ ]:
# fifth, add the surrogate key to the data table
data = data.merge(business_dim,
                  left_on = unique_row,
                  right_on = unique_row,
                  how = 'left')

data.head(100)

### Step 6: Creating Fact table

In [ ]:
data.columns

In [ ]:
# take a subset of fact_table for only the needed columns: which are keys and measures
fact_table = data[['business_id',
              'location_id',
              'swc_sq_ft',
               'swc_tables',
               'swc_chairs']]

fact_table

### Step 6: Deliver Facts and Dimensions to Data Warehouse (BigQuery)

In [ ]:
# create a function to load dataframes to BigQuery

def load_table_to_bigquery(df,
                          table_name,
                          dataset_id):

    dataset_id = dataset_id #change 301800 to match your project id

    dataset_ref = bigquery_client.dataset(dataset_id)
    job_config = bigquery.LoadJobConfig()
    job_config.autodetect = True
    job_config.write_disposition = "WRITE_TRUNCATE"

    upload_table_name = f"{dataset_id}.{table_name}"
    
    load_job = bigquery_client.load_table_from_dataframe(df,
                                                upload_table_name,
                                                job_config = job_config)
        
    print(f"""completed loading {table_name} --
         {load_job}""")

In [ ]:
load_table_to_bigquery(df = location_dim,
                       table_name = "location_dim",
                       dataset_id = dataset_id)

In [ ]:
load_table_to_bigquery(df = business_dim,
                       table_name = "business_dim",
                       dataset_id = dataset_id)

In [ ]:
load_table_to_bigquery(df = fact_table,
                       table_name = "sidewalk_fact",
                       dataset_id = dataset_id)